In [50]:
import findspark
import numpy as np
from pyspark import StorageLevel

findspark.init()
findspark.find()

'C:\\spark-3.0.1-bin-hadoop2.7'

In [51]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [52]:
%%time

#initialization of spark session
sk = SparkSession.builder.appName("RDD_NBA_stats").getOrCreate()

#load the datasets
df = sk.read.format("csv").option("header", "true").load("NBA_Data/rs_10-20.csv")



Wall time: 225 ms


In [53]:
df.persist(StorageLevel.MEMORY_ONLY)
df.is_cached

True

In [54]:
df

DataFrame[ID: string, TEAM: string, GP: string, W: string, L: string, WIN%: string, MIN: string, PTS: string, FGM: string, FGA: string, FG%: string, 3PM: string, 3PA: string, 3P%: string, FTM: string, FTA: string, FT%: string, OREB: string, DREB: string, REB: string, AST: string, TOV: string, STL: string, BLK: string, BLKA: string, PF: string, PFD: string, +/-: string, SEASN: string]

In [55]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- TEAM: string (nullable = true)
 |-- GP: string (nullable = true)
 |-- W: string (nullable = true)
 |-- L: string (nullable = true)
 |-- WIN%: string (nullable = true)
 |-- MIN: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- FGM: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- FG%: string (nullable = true)
 |-- 3PM: string (nullable = true)
 |-- 3PA: string (nullable = true)
 |-- 3P%: string (nullable = true)
 |-- FTM: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- FT%: string (nullable = true)
 |-- OREB: string (nullable = true)
 |-- DREB: string (nullable = true)
 |-- REB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- BLKA: string (nullable = true)
 |-- PF: string (nullable = true)
 |-- PFD: string (nullable = true)
 |-- +/-: string (nullable = true)
 |-- SEASN: strin

In [56]:
%%time
# Query1 
# Each Team Win and Lost Count?
# first gather all the data 

rdd = df.rdd
rdd = rdd.map(lambda x: (x[1],x[3], x[4]))\
.sortBy(lambda x: x[1], ascending=False)
rdd.take(10)
    


Wall time: 679 ms


[('Golden State Warriors', '73', '9'),
 ('Charlotte Bobcats', '7', '59'),
 ('Golden State Warriors', '67', '15'),
 ('San Antonio Spurs', '67', '15'),
 ('Golden State Warriors', '67', '15'),
 ('Miami Heat', '66', '16'),
 ('Houston Rockets', '65', '17'),
 ('Chicago Bulls', '62', '20'),
 ('San Antonio Spurs', '62', '20'),
 ('San Antonio Spurs', '61', '21')]

In [57]:
%%time
# Query 2 reduce them
rdd = rdd.map(lambda x: (x[0] , [float(x[1]) , float(x[2]) , 1]))\
.reduceByKey(lambda x,y : [x[0] + y[0] , x[1] + y[1],x[2] + y[2]])\
.map(lambda x: [x[0],x[1][0] /x[1][2] ,x[1][1]])\
.sortBy(lambda x: x[1], ascending=False)
rdd.collect()

Wall time: 1.59 s


[['San Antonio Spurs', 54.1, 252.0],
 ['Oklahoma City Thunder', 50.9, 285.0],
 ['Golden State Warriors', 49.4, 293.0],
 ['Houston Rockets', 49.0, 304.0],
 ['LA Clippers', 48.6, 157.0],
 ['Los Angeles Clippers', 48.2, 153.0],
 ['Miami Heat', 47.7, 318.0],
 ['Boston Celtics', 45.4, 339.0],
 ['Toronto Raptors', 45.3, 341.0],
 ['Indiana Pacers', 45.0, 344.0],
 ['Portland Trail Blazers', 43.6, 360.0],
 ['Denver Nuggets', 43.0, 365.0],
 ['Memphis Grizzlies', 42.2, 373.0],
 ['Utah Jazz', 41.4, 380.0],
 ['Chicago Bulls', 40.9, 378.0],
 ['Dallas Mavericks', 40.8, 389.0],
 ['Milwaukee Bucks', 39.5, 400.0],
 ['Atlanta Hawks', 39.0, 399.0],
 ['New Orleans Pelicans', 36.285714285714285, 310.0],
 ['Charlotte Hornets', 35.833333333333336, 260.0],
 ['Los Angeles Lakers', 35.8, 435.0],
 ['Washington Wizards', 35.2, 442.0],
 ['Brooklyn Nets', 34.625, 369.0],
 ['Cleveland Cavaliers', 34.6, 441.0],
 ['Philadelphia 76ers', 33.1, 464.0],
 ['Detroit Pistons', 32.6, 462.0],
 ['Orlando Magic', 32.1, 474.0],
 [

In [58]:
# Query 3
# 1. Can be (the best most field GOAL) guarantee to be (the best winner)?
max_field_goal_item = rdd.sortBy(lambda x: x[1], ascending=False).take(1)

max_field_goal , win_percent = max_field_goal_item[0][1] , max_field_goal_item[0][2]
print("max field goal : " + str(max_field_goal) + "\n"
      + "its max win percent : " + str(win_percent))

q1rdd = rdd.filter(lambda x: x[2] >= win_percent)\
.sortBy(lambda x: x[2], ascending=False)


if(len(q1rdd.collect()) >0 ):
    print("\nThere are some teams which their field goal is less than max field goal\nSo, it doesn't guarantee\n")
else:
     print("\nIt guarantees\n")
q1rdd.collect()

max field goal : 54.1
its max win percent : 252.0

There are some teams which their field goal is less than max field goal
So, it doesn't guarantee



[['Sacramento Kings', 29.3, 501.0],
 ['Minnesota Timberwolves', 29.2, 494.0],
 ['Phoenix Suns', 30.6, 489.0],
 ['Orlando Magic', 32.1, 474.0],
 ['New York Knicks', 31.6, 472.0],
 ['Philadelphia 76ers', 33.1, 464.0],
 ['Detroit Pistons', 32.6, 462.0],
 ['Washington Wizards', 35.2, 442.0],
 ['Cleveland Cavaliers', 34.6, 441.0],
 ['Los Angeles Lakers', 35.8, 435.0],
 ['Milwaukee Bucks', 39.5, 400.0],
 ['Atlanta Hawks', 39.0, 399.0],
 ['Dallas Mavericks', 40.8, 389.0],
 ['Utah Jazz', 41.4, 380.0],
 ['Chicago Bulls', 40.9, 378.0],
 ['Memphis Grizzlies', 42.2, 373.0],
 ['Brooklyn Nets', 34.625, 369.0],
 ['Denver Nuggets', 43.0, 365.0],
 ['Portland Trail Blazers', 43.6, 360.0],
 ['Indiana Pacers', 45.0, 344.0],
 ['Toronto Raptors', 45.3, 341.0],
 ['Boston Celtics', 45.4, 339.0],
 ['Miami Heat', 47.7, 318.0],
 ['New Orleans Pelicans', 36.285714285714285, 310.0],
 ['Houston Rockets', 49.0, 304.0],
 ['Golden State Warriors', 49.4, 293.0],
 ['Oklahoma City Thunder', 50.9, 285.0],
 ['Charlotte Hor

In [83]:
%%time
# Query 4
# group by data (Win)
rdd = df.rdd.cache()
rdd = rdd.map(lambda x: (x[0],x[1],x[2],x[5],x[7],x[28]))\
.groupBy(lambda x: x[3])\
.mapValues(list)

rdd.collect()




Wall time: 1.47 s


[('0.756',
  [('1', 'Chicago Bulls', '82', '0.756', '98.6', "'10-11'"),
   ('1', 'San Antonio Spurs', '82', '0.756', '105.4', "'13-14'")]),
 ('0.744',
  [('2', 'San Antonio Spurs', '82', '0.744', '103.7', "'10-11'"),
   ('2', 'San Antonio Spurs', '82', '0.744', '105.3', "'16-17'")]),
 ('0.707',
  [('3', 'Miami Heat', '82', '0.707', '102.1', "'10-11'"),
   ('3', 'San Antonio Spurs', '82', '0.707', '103', "'12-13'"),
   ('3', 'Golden State Warriors', '82', '0.707', '113.5', "'17-18'"),
   ('2', 'Toronto Raptors', '82', '0.707', '114.4', "'18-19'")]),
 ('0.695',
  [('4', 'Dallas Mavericks', '82', '0.695', '100.2', "'10-11'"),
   ('4', 'Los Angeles Lakers', '82', '0.695', '101.5', "'10-11'"),
   ('4', 'Denver Nuggets', '82', '0.695', '106.1', "'12-13'"),
   ('3', 'Los Angeles Clippers', '82', '0.695', '107.9', "'13-14'"),
   ('3', 'Cleveland Cavaliers', '82', '0.695', '104.3', "'15-16'"),
   ('3', 'Golden State Warriors', '82', '0.695', '117.7', "'18-19'")]),
 ('0.683',
  [('6', 'Boston Ce

In [84]:
%%time
# Query 5
# group by data Team Name
rdd = df.rdd.cache()
rdd = rdd.map(lambda x: (x[0],x[1]))\
.groupBy(lambda x: x[1])\
.mapValues(list)

rdd.collect()

Wall time: 1.47 s


[('Chicago Bulls',
  [('1', 'Chicago Bulls'),
   ('1', 'Chicago Bulls'),
   ('11', 'Chicago Bulls'),
   ('11', 'Chicago Bulls'),
   ('9', 'Chicago Bulls'),
   ('14', 'Chicago Bulls'),
   ('15', 'Chicago Bulls'),
   ('24', 'Chicago Bulls'),
   ('27', 'Chicago Bulls'),
   ('24', 'Chicago Bulls')]),
 ('San Antonio Spurs',
  [('2', 'San Antonio Spurs'),
   ('1', 'San Antonio Spurs'),
   ('3', 'San Antonio Spurs'),
   ('1', 'San Antonio Spurs'),
   ('5', 'San Antonio Spurs'),
   ('2', 'San Antonio Spurs'),
   ('2', 'San Antonio Spurs'),
   ('12', 'San Antonio Spurs'),
   ('11', 'San Antonio Spurs'),
   ('19', 'San Antonio Spurs')]),
 ('Miami Heat',
  [('3', 'Miami Heat'),
   ('4', 'Miami Heat'),
   ('1', 'Miami Heat'),
   ('5', 'Miami Heat'),
   ('21', 'Miami Heat'),
   ('7', 'Miami Heat'),
   ('15', 'Miami Heat'),
   ('15', 'Miami Heat'),
   ('17', 'Miami Heat'),
   ('11', 'Miami Heat')]),
 ('Dallas Mavericks',
  [('4', 'Dallas Mavericks'),
   ('13', 'Dallas Mavericks'),
   ('17', 'Dallas 